In [ ]:
import sys
import unittest, time, re,os,threading
from dateutil import parser
import pandas as pd
import itertools
import glob
from settings import TWITTER_USERNAME,TWITTER_PASSWORD

In [ ]:
hashtagProMahasiswa=["#kpkkuatkorupsiturun", "#KekuasaanDitanganRakyat", "#GejayanMemanggil2", "#diperkosanegara", "#mahasiswabergerak", "#hidupmahasiswa", "#tolakruukuhp", "#tolakrevisiuukpk", "#hiduprakyatindonesia", "#mositidakpercaya", "#reformasidikorupsi", "#stmmelawan", "#stmbergerak", "#gejayanmemanggil", "#indonesiaberduka", "#bebaskananandabadudu", "#BebaskanDandhyLaksono", "#savekpk", "#petisisavekpk","#belajardaripelajar","#rakyatbergerak","#revisiuukpkfornkri", "#dukungrevisiuukpk", "#kpkcengeng", "#tempokacungkpk", "#kpkpatuhaturan", "#kpklebihbaik", "#sayabersamajokowi", "#jokowimendengarrakyat", "#kitadukungjokowi","#percayalangkahjokowi"]
hashtagProMahasiswa = [item.lower() for item in hashtagProMahasiswa]

In [ ]:
all_files = glob.glob("*.csv")
li = []
for filename in all_files:
   df = pd.read_csv(filename, index_col=None, header=0)
   li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('combined.csv')


In [ ]:
df = pd.read_csv("combined.csv")
df.columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1','time','extraInfoTargetUser', 'extraInfoContent','content', 'lang', 'mentions','contentUrl'],axis =1)
print("Shape : "+str(df.shape)) #overall shape
#remove duplicate column
dfDrop = df.copy(deep=True)
dfDrop = dfDrop.drop_duplicates(subset="urlTweet")
print("Dropped Shape : "+str(dfDrop.shape))
#remove empty column
dfDrop = dfDrop.dropna(subset=['hashtags'])
print("Empty Row Dropped Shape : "+str(dfDrop.shape))
indexNames = dfDrop[ dfDrop['hashtags'] == "[]" ].index
dfDrop.drop(indexNames,inplace =True)
print("Empty Row [] Dropped Shape : "+str(dfDrop.shape))
dfDrop['hashtags']= dfDrop['hashtags'].str.lower()
dfDrop.head(5)
dfDrop.to_csv("processed.csv")

In [ ]:
#example
# d={}
# d[('Antje','place')]='Barcelona'
# d[('Antje','year')]=1987
# d[('Antje','YEAR')]=1988
# d[('Mike','place')]='Berlin'
# key=('Antje','year')
# print(d)
# if key in d:
#     d[key]+=1
# else:
#     d[key]='a'
# print(d)

In [ ]:
#accumulate interaction method
myDict={}
x = 0
for index,row in dfDrop.iterrows():
    ht = row['hashtags'].strip('[')
    ht = ht.strip(']')
    ht = ht.replace("'","")
    ht = ht.replace(" ","")
    htArr = ht.split(',')
    for h in htArr:
        if h not in hashtagProMahasiswa:
            break;
        else:
            key = (row['username'],h)
            if key in myDict:
                myDict[key]+=1
            else:
                myDict[key]=1
for t in myDict:
    print(t[0])
    print(t[1])
    print(myDict[t])


In [ ]:
hashtagCount = len(hashtagProMahasiswa)
usernameCount = dfDrop.username.unique().shape[0]
usernames = dfDrop.username.unique()
usernameDict = {}
hashtagDict = {}
file = open("csvToPajek.net","w")
file.write("*vertices     %d %d\n"%(hashtagCount+len(usernames),len(usernames)))
#write users
for x in range(len(usernames)):
    usernameDict[usernames[x]]=x+1
    file.write("     %d \"%s\"\n"%(x+1,usernames[x]))
#write hashtags
for x in range(hashtagCount):
    hashtagDict[hashtagProMahasiswa[x]]=x+len(usernames)+1
    file.write("     %d \"%s\"\n"%(x+len(usernames)+1,hashtagProMahasiswa[x]))
file.write("*Edges\n")
#write Edges
for x in myDict:
    userNameId = usernameDict[x[0]]
    hashTagId = hashtagDict[x[1]]
    weight = myDict[x]
    file.write("     %s  %s   %d\n"%(userNameId,hashTagId,weight))
file.close()

In [ ]:
_source=[]
_target=[]
_weight=[]
_id=[]
gephiDict = {
    "ID":_id,
    "Source":_source,
    "Target":_target,
    "Weight":_weight,
}
i = 1
for x in myDict:
    _id.append(i)
    _source.append(x[0])
    _target.append(x[1])
    _weight.append(myDict[x])
    i+=1
gephi = pd.DataFrame.from_dict(gephiDict,orient='index').T 
gephi.to_csv("gephiNew.csv",index=False)